In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import os
import sys
from pathlib import Path
import timm
import json

In [ ]:
# make dataset directory visible as a module
modules_path = os.path.abspath("..")
if modules_path not in sys.path:
    sys.path.append(modules_path)

# import dataset
from data.FloodNet import FloodNetDataset

In [ ]:
# setup computing on CUDA if available
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

current_cuda_device = None
print(f"is cuda available: {torch.cuda.is_available()}")
print(f"cuda device count: {torch.cuda.device_count()}")
current_cuda_device = torch.cuda.current_device()
print(f"current cuda device: {current_cuda_device}")
print(f"current cuda device name: {torch.cuda.get_device_name(current_cuda_device)}")

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

In [ ]:
# image transforms pipeline
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize with ImageNet mean and std
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# create train & test dataset objects
with open("../data/classes.json", "r") as j:
    classes = json.load(j)

# Create train dataset
train_images_dir = Path("../data/FloodNet_dataset/train/image")
train_csv_file = Path("../data/flood_train_rel_paths.csv")
train_data = FloodNetDataset(annotations_file=train_csv_file, img_dir=train_images_dir, classes=classes, transform=transform)

# Create test dataset
test_images_dir = Path("../data/FloodNet_dataset/test/image")
test_csv_file = Path("../data/flood_test_rel_paths.csv")
test_data = FloodNetDataset(annotations_file=test_csv_file, img_dir=test_images_dir, classes=classes, transform=transform)

In [ ]:
# create dataloaders
BATCH_SIZE = 64

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)